### Импортируем библиотеки

In [1]:
import csv
import time
import requests
import pandas as pd
from pandas.tseries.offsets import DateOffset

### Назначим глобальные переменные

In [10]:
token = 'Введите сюда свой token'
version = 5.131
count = 100

### Напишем функцию парсинга постов на стене (метод 'wall.get')

In [3]:
def take_all_posts():
    domain = '1tsprint'
    all_posts = []
    offset = 0
    while offset < 1000:
        response = requests.get('https://api.vk.com/method/wall.get',
                                params={
                                    'access_token': token,
                                    'v': version,
                                    'domain': domain,
                                    'count': count,
                                    'offset': offset
                                    }
                                )
        data = response.json()['response']['items']
        offset += 100
        all_posts.extend(data)
        time.sleep(0.5)
    return all_posts
all_posts = take_all_posts()

### Вытянем из таблицы данные о количестве лайков и количестве комментариев.

In [4]:
list_posts, list_likes, list_comments, list_date_of_post= [], [], [], []
for post in all_posts:
    list_posts.append(post['text'])
    list_likes.append(post['likes']['count'])
    list_comments.append(post['comments']['count'])
    list_date_of_post.append(post['date'])  
list_date_of_post = pd.Series(list_date_of_post)   

count_df = pd.concat([pd.Series(list_posts), pd.Series(list_likes), pd.Series(list_comments), list_date_of_post.astype('int').astype("datetime64[s]")], axis=1)
count_df.columns = ['text_post', 'count_likes', 'count_comments', 'post_date']
count_df.head()

,text_post,count_likes,count_comments,post_date
0,Что такое Zoom-этикет \n \n💻 Видеоконференции ...,4,0,2023-07-16 12:45:00
1,Мастермайнд: как находить лучшие решения с пом...,2,0,2023-07-15 12:30:00
2,Заключительная часть советов от участников раз...,7,2,2023-07-14 13:04:05
3,Как понимать корпоративный сленг \n \n💬 У нас ...,5,0,2023-07-13 13:45:00
4,Как в 1Т Клубе организована менторская поддерж...,15,0,2023-07-12 13:00:01


### Найдем сегодняшнее число и отнимем от этой даты 3 месяца, чтобы вывести TOP 10 самых популярных новостей за последние 3 месяца.

In [5]:
date_today = pd.Timestamp.now().normalize()
date_today

Timestamp('2023-07-17 00:00:00')

In [6]:
find_date = pd.Timestamp.now().normalize() - DateOffset(months = 3)
find_date

Timestamp('2023-04-17 00:00:00')

### Выведем все значения таблицы за последние 3 месяца.

In [7]:
df_3_last_month = count_df.loc[(count_df['post_date'] >= find_date)]
df_3_last_month.tail()

,text_post,count_likes,count_comments,post_date
96,География 1Т — итоги конкурса \n \n🏙 В субботу...,8,0,2023-04-19 14:22:50
97,Бесплатные курсы программирования для школьник...,9,0,2023-04-19 10:51:59
98,Я тоже хочу на хакатон! \nНейросети и искусств...,6,0,2023-04-18 14:15:54
99,Приглашаем родителей школьников 8–11 классов н...,2,0,2023-04-18 07:42:07
100,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,44,3,2023-04-17 13:48:06


### Выведем ТОП 10 самых популярных новостей за последние 3 месяца (популярность новости определяется количеством лайков). Сохраним файл в формате *.csv.

In [8]:
df_popular_likes = df_3_last_month.nlargest(n=10, columns=['count_likes'], keep="all")[['text_post', 'count_likes']]
df_popular_likes.to_csv('top10_popular_posts.csv')
df_popular_likes

,text_post,count_likes
100,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,44
15,1Т будет участвовать в развитии беспилотного т...,20
95,Проекты Спринта: сайт для РСО \n \n🙎‍♂ Значите...,20
91,Проекты-победители 7-го потока \n \n🏆 Презента...,19
66,🐭 У нас появился классный мерч — коврики и ком...,18
86,Профессия аналитик: открытый вебинар,18
29,Как создавались курсы 1Т? \n \nСначала мы обза...,17
73,1Т — спонсор конкурса Мисс и Мистер СПбСО 2023...,17
87,🚀 Три новых курса от 1Т по искусственному инте...,17
89,🧙‍♀ Выложили фотоотчет с самого масштабного Оф...,16


### Выведем TOP 10 постов с комментариями (выводятся посты с максимальным количеством комментариев). По условию не сказано, за какой период, поэтому выведем ТОП10 за все время существования группы VK. Сохраним файл в формате *.csv.

In [9]:
df_popular_comments = count_df.nlargest(n=10, columns=['count_comments'], keep="all")[['text_post', 'count_comments']]
df_popular_comments.to_csv('top10_popular_comments.csv') 
df_popular_comments

,text_post,count_comments
311,Еще три новых термина 🎉 \n \nДва слова из трех...,7
145,Приручаем искусственный интеллект \n \n🎨 1 мар...,5
38,Итоги конкурса #навыки1Т \n \n🐭 Недавно у 1Т С...,3
100,Дорогие друзья! \n \n🎉 Недавно четырнадцать ко...,3
2,Заключительная часть советов от участников раз...,2
123,"🏴‍☠ Выложили на нашем YouTube-канале видео, то...",2
13,"7 полезных сервисов для тех, кто работает с те...",1
33,Чего не стоит делать на собеседовании \n \nВы ...,1
140,🔔 Свистать всех наверх! \n \n🏴‍☠ ИТ-конференци...,1
190,Как работает Bluetooth \n \n⚡ Bluetooth — техн...,1
